## Flood Depth Estimation
### This notebook takes a flood extent raster file and estimates flood depth using a specified DEM
#### Output
- Flood extent raster with depth estimates
#### Input
- Flood extent raster (without depth)
- DEM
#### Method summary
- Clip the DEM and supplied flood extent raster to the domain mask
- Regrid the supplied flood extent raster so it has the same resolution and coordinate system as the DEM (each should have same number of rows and cols)
- For each contiguous "blob" in the flood extent raster:
  - Find the edge pixels
  - Assign the DEM values to the edge pixels
  - Interpolate a surface using an iterative circular window (mean, max, or percentile options are included)
  - Subtract the DEM from the interpolated surface to the depth estimates
  - Save final layer to disk

In [ ]:
from scipy.ndimage import label, generate_binary_structure, binary_erosion, generic_filter, iterate_structure
import rioxarray as rxr
import geopandas as gpd
import numpy as np
from numba import njit, jit
import rasterio

In [ ]:
%%time
# Set filepaths
DOMAIN_MASK_FILEPATH = "/Users/slamont/sar_flood_extent/mask_geoms.geojson" 
DEM_FILEPATH = "/Users/slamont/sar_flood_extent/dem/sample_dem.tif"
FLOOD_EXTENT_FILEPATH = "/Users/slamont/sar_flood_extent/mapped_flood_extent.tif"
OUTPUT_EXTENT_WITH_DEPTH = "/Users/slamont/sar_flood_extent/mapped_flood_extent_with_depth.tif"

da_dem = rxr.open_rasterio(DEM_FILEPATH)
da_extent = rxr.open_rasterio(FLOOD_EXTENT_FILEPATH)

gdf_all = gpd.read_file(DOMAIN_MASK_FILEPATH)
mask_geom = gdf_all.geometry.tolist()  # NOTE: Assumes a single geometry
da_dem = da_dem.rio.clip(mask_geom)
da_extent = da_extent.rio.clip(mask_geom)

# Align the flood extent layer with the DEM in terms of resolution and coordinate system
da_extent = da_extent.rio.reproject_match(da_dem)

In [ ]:
%%time
s = generate_binary_structure(2,2)

# Create binary array
arr_obs = da_extent.values[0]
arr_obs[arr_obs > 0] = 1
arr_obs[arr_obs <= 0] = 0

# Label the blobs
labeled_arr, num_feats = label(arr_obs, structure=s)

# Get the edges of blobs
mask = binary_erosion(labeled_arr.tolist())
edge_arr = labeled_arr.copy()
edge_arr[mask] = 0

arr_dem = da_dem.values[0]
arr_dem[arr_dem == -9999] = np.nan

arr_out = np.full(arr_dem.shape, fill_value=np.nan, dtype=np.float32)

WINDOW_SIZE = 9
PERCENTILE = 60

s = generate_binary_structure(2,1)
FOOTPRINT = iterate_structure(s, 4)
weights = np.ones((5, 5))

In [ ]:
%%time
@njit()
def window_nanmean(x):
    return np.nanmean(x)

@njit()
def window_nanmax(x):
    return np.nanmax(x)

@njit()
def window_nanperc(x):
    return np.nanpercentile(x, PERCENTILE)

# Loop over blobs
for feat in range(2, num_feats):
        
    # Get blob and edge indices
    blob_inds = np.where(labeled_arr == feat)
    edge_inds = np.where(edge_arr == feat)
    
    if blob_inds[0].size == edge_inds[0].size:
        arr_out[blob_inds] = arr_dem[blob_inds]
        continue
    
    # Create normalized arrays for this blob
    min_row = edge_inds[0].min()
    max_row = edge_inds[0].max()
    
    min_col = edge_inds[1].min()
    max_col = edge_inds[1].max()
    
    rows = (max_row + 1) - min_row
    cols = (max_col + 1) - min_col
    
    # Get a temporary edge pixel array with DEM values
    arr_tmp_edge = np.full((rows, cols), fill_value=np.nan, dtype=np.float32)
    relative_inds = (edge_inds[0] - min_row, edge_inds[1] - min_col)
    arr_tmp_edge[relative_inds] = arr_dem[edge_inds]   
    
    # Run the filter to generate water elevation surface
    out = generic_filter(arr_tmp_edge, window_nanperc, footprint=FOOTPRINT) 
    while out[np.isnan(out)].size > 2:
        # print(i)
        out = generic_filter(arr_tmp_edge, window_nanperc, footprint=FOOTPRINT)  
        arr_tmp_edge = out.copy()

    # Reset edge pixel vals
    mask = ~np.isnan(arr_tmp_edge)
    out[mask] = arr_tmp_edge[mask]
    
    # Now mask out pixels exterior to the blob
    arr_tmp_blob = np.full((rows, cols), fill_value=np.nan, dtype=np.float32)
    relative_inds = (blob_inds[0] - min_row, blob_inds[1] - min_col)
    arr_tmp_blob[relative_inds] = arr_dem[blob_inds]    
    out[np.isnan(arr_tmp_blob)] = np.nan
    
    # Finally assign the filtered window to the main output array
    inds_notnan = np.where(~np.isnan(out))
    abs_inds = inds_notnan[0] + min_row, inds_notnan[1] + min_col
    arr_out[abs_inds] = out[inds_notnan] - arr_dem[abs_inds]

# Save final flood extent layer with depth estimates to disk
da_out = da_extent.copy()
da_out[:, :] = arr_out
da_out.rio.to_raster(OUTPUT_EXTENT_WITH_DEPTH)